In [1]:
import cv2 as cv
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
img_height = 180
img_width = 180

In [3]:
import numpy as np
import pywt
import cv2    

def w2d(img, mode='haar', level=1):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)   
    imArray /= 255;
    # compute coefficients 
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)  
    coeffs_H[0] *= 0;  

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H

In [4]:
path_to_data = "./cropped_images/"

In [5]:
import os
img_dirs = []
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        img_dirs.append(entry.path)
img_dirs

['./cropped_images/chris_evans',
 './cropped_images/chris_hemsworth',
 './cropped_images/mark_ruffalo',
 './cropped_images/robert_downey_jr',
 './cropped_images/scarlett_johansson']

In [6]:
celebrity_file_names_dict = {}
for img_dir in img_dirs:
    count = 1
    celebrity_name = img_dir.split('/')[-1]
    print(celebrity_name)
    celebrity_file_names_dict[celebrity_name] = []
    
    for entry in os.scandir(img_dir):
        celebrity_file_names_dict[celebrity_name].append(entry.path)

chris_evans
chris_hemsworth
mark_ruffalo
robert_downey_jr
scarlett_johansson


In [7]:
class_dict = {}
count = 0
for celebrity_name in celebrity_file_names_dict.keys():
    class_dict[celebrity_name] = count
    count = count + 1
class_dict

{'chris_evans': 0,
 'chris_hemsworth': 1,
 'mark_ruffalo': 2,
 'robert_downey_jr': 3,
 'scarlett_johansson': 4}

In [8]:
X, y = [], []
for celebrity_name, training_files in celebrity_file_names_dict.items():
    for training_image in training_files:
        img = cv2.imread(training_image)
        if img is None:
            continue
        scalled_raw_img = cv2.resize(img, (img_height, img_width))
#         img_har = w2d(img,'db1',5)
#         scalled_img_har = cv2.resize(img_har, (32, 32))
#         combined_img = np.vstack((scalled_raw_img.reshape(32*32*3,1),scalled_img_har.reshape(32*32,1)))
        X.append(scalled_raw_img)
        y.append(class_dict[celebrity_name])

In [9]:
X[0].shape

(180, 180, 3)

In [10]:
X = np.array(X)
X.shape

(274, 180, 180, 3)

In [11]:
X = X/255

In [12]:
y = np.array(y)
type(y)

numpy.ndarray

In [13]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


In [15]:
type(X_train[0])

numpy.ndarray

In [20]:
model =keras.Sequential([
  keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(5)
])

In [25]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 180, 180, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 180, 180, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 90, 90, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 90, 90, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 45, 45, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 45, 45, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 22, 64)       

In [26]:
epochs=10
history = model.fit(
  X_train,
  y_train,
  epochs=epochs
)

Epoch 1/10
7/7 [==============================] - 1s 109ms/step - loss: 6.4896 - accuracy: 0.2678
Epoch 2/10
7/7 [==============================] - 1s 108ms/step - loss: 6.1724 - accuracy: 0.2069
Epoch 3/10
7/7 [==============================] - 1s 109ms/step - loss: 6.2662 - accuracy: 0.2470
Epoch 4/10
7/7 [==============================] - 1s 109ms/step - loss: 6.1190 - accuracy: 0.2794
Epoch 5/10
7/7 [==============================] - 1s 106ms/step - loss: 6.0633 - accuracy: 0.2537
Epoch 6/10
7/7 [==============================] - 1s 106ms/step - loss: 5.7179 - accuracy: 0.1955
Epoch 7/10
7/7 [==============================] - 1s 109ms/step - loss: 5.8810 - accuracy: 0.2101
Epoch 8/10
7/7 [==============================] - 1s 109ms/step - loss: 5.9615 - accuracy: 0.2473
Epoch 9/10
7/7 [==============================] - 1s 108ms/step - loss: 5.3815 - accuracy: 0.2783
Epoch 10/10
7/7 [==============================] - 1s 105ms/step - loss: 6.3702 - accuracy: 0.2209


In [ ]:
256//2

In [27]:
cnn = keras.Sequential(
[
    # cnn
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(img_height,img_height,3)),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    
    # Dense
    keras.layers.Flatten(),
    keras.layers.Dense(128 ,activation='relu'),
       
    keras.layers.Dense(5,  activation='softmax')
])
loss_fn = keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction="auto", name="sparse_categorical_crossentropy"
)
cnn.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
cnn.fit(X_train, y_train, epochs=40)

Epoch 1/40
7/7 [==============================] - 5s 366ms/step - loss: 3.3153 - accuracy: 0.2538
Epoch 2/40
7/7 [==============================] - 1s 176ms/step - loss: 1.6105 - accuracy: 0.1988
Epoch 3/40
7/7 [==============================] - 1s 175ms/step - loss: 1.5562 - accuracy: 0.2479
Epoch 4/40
7/7 [==============================] - 1s 174ms/step - loss: 1.4120 - accuracy: 0.4529
Epoch 5/40
7/7 [==============================] - 1s 175ms/step - loss: 1.1070 - accuracy: 0.6609
Epoch 6/40
7/7 [==============================] - 1s 175ms/step - loss: 0.7565 - accuracy: 0.7534
Epoch 7/40
7/7 [==============================] - 1s 175ms/step - loss: 0.6675 - accuracy: 0.8004
Epoch 8/40
7/7 [==============================] - 1s 178ms/step - loss: 0.5133 - accuracy: 0.8491
Epoch 9/40
7/7 [==============================] - 1s 178ms/step - loss: 0.2790 - accuracy: 0.8931
Epoch 10/40
7/7 [==============================] - 1s 180ms/step - loss: 0.2167 - accuracy: 0.9469
Epoch 11/40
7/7 [==

In [28]:
cnn.evaluate(X_test, y_test)

3/3 [==============================] - 1s 328ms/step - loss: 1.4799 - accuracy: 0.7681


[1.4799182415008545, 0.7681159377098083]

In [ ]:
y_pred = cnn.predict(x=X_test)

In [ ]:
y_clsses = [np.argmax(e) for e in y_pred]
y_clsses[:5]

In [ ]:
y_test[:5]